In [1]:
import pandas as pd
import numpy as np
import os

import re
import emoji
from konlpy.tag import Okt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader -> datasets 라이브러리랑 충돌

c:\Users\ehddl\AppData\Local\pypoetry\Cache\virtualenvs\sns-categorizer-wO1G7-CE-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/")

In [3]:
new_data = pd.read_parquet("media_data_sample.parquet")

In [5]:
new_data_profile = pd.read_parquet("C:/Users/ehddl/Downloads/merged_data.parquet")

In [6]:
new_data_profile = new_data_profile[['acnt_id', 'acnt_sub_nm', 'intro_txt']]

In [ ]:
# new_data = new_data[['media_cn']]

In [7]:
new_data = new_data[['acnt_id', 'media_cn']]

data preprocessing

In [8]:
new_data_profile.dropna(inplace=True)

In [9]:
new_data.dropna(inplace=True)

In [10]:
new = pd.merge(new_data_profile, new_data, on='acnt_id')
new

,acnt_id,acnt_sub_nm,intro_txt,media_cn
0,17841400005571149,Yjmgoslow,YJM*,YJM* will Drop a new album in the hottest summ...
1,17841400005571149,Yjmgoslow,YJM*,yg
2,17841400005571149,Yjmgoslow,YJM*,저의 근황은....더보기
3,17841400005571149,Yjmgoslow,YJM*,99년식 김윤호 생일 축하해 주셔서 다들 감사해요💛
4,17841400005571149,Yjmgoslow,YJM*,Yoooooooo
...,...,...,...,...
16666,17841465962045202,kikunionni,키쿠니온니 🍑\n오전엔 직장인 오후엔 초보사장 (ว˙o˙)ว\n3잡러의 일상 기록🙂‍↔️,오늘도 너무 더워요('-')\n자르고 또 자르고\n\n요즘 중 단발이 유행이라\n다...
16667,17841465962045202,kikunionni,키쿠니온니 🍑\n오전엔 직장인 오후엔 초보사장 (ว˙o˙)ว\n3잡러의 일상 기록🙂‍↔️,요즘 정말 덥지 않나요..?\n녹아내릴 것 같은 날씨🔥\n\n저는 퇴근하고\n시원하...
16668,17841465962045202,kikunionni,키쿠니온니 🍑\n오전엔 직장인 오후엔 초보사장 (ว˙o˙)ว\n3잡러의 일상 기록🙂‍↔️,인님들💖\n사실 저 6월부터\n새로운 직장으로 이직했어요!\n\n경기도민이 서울로 ...
16669,17841465962045202,kikunionni,키쿠니온니 🍑\n오전엔 직장인 오후엔 초보사장 (ว˙o˙)ว\n3잡러의 일상 기록🙂‍↔️,일만 하지 말고\n잠깐 시간 내어\n놀기도 해야죠🕶\n\n자꾸 같이 다니고 싶다던\...


In [11]:
def clean_text(text):
    if not isinstance(text, str):
        return ''
    
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [12]:
new['acnt_sub_nm_cleaned'] = new['acnt_sub_nm'].apply(clean_text)
new['intro_txt_cleaned'] = new['intro_txt'].apply(clean_text)
new['media_cn_cleaned'] = new['media_cn'].apply(clean_text)

In [13]:
new = new[~new.apply(lambda row: row.astype(str).str.strip().eq('').any(), axis=1)]

In [14]:
new = new[['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']]

In [15]:
num_labels = 26
predict_dataset = Dataset.from_pandas(new)

single model prediction

In [12]:
# model_name_1 = "kykim/bert-kor-base"
# model_path_1 = "finetune-bert-kor"

# model1 = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
# model1_tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
# model1.eval()

In [13]:
# model_name_2 = "BM-K/KoSimCSE-roberta"
# model_path_2 = "finetune-BM-K"

# model2 = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
# model2_tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
# model2.eval()

In [16]:
model_name = "kykim/bert-kor-base"
model_path = "muli-columns-kykim-bert-kor"

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
model_tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [17]:
predict_dataset = Dataset.from_pandas(new)

In [ ]:
# def tokenize_fn_for_prediction(ex):
#     # label 칼럼이 없으므로 text 컬럼만 토큰화 진행
#     return model_tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=512) # max_length 조정 (128->512)

# predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
# predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"]) # 라벨은 없으므로 제외

Map: 100%|██████████| 18282/18282 [00:01<00:00, 12033.08 examples/s]


In [18]:
def tokenize_three_columns(examples):
    combined_texts = [
        f"{acnt} {model_tokenizer.sep_token} {intro} {model_tokenizer.sep_token} {txt}"
        for acnt, intro, txt in zip(
            examples["acnt_sub_nm_cleaned"],
            examples["intro_txt_cleaned"],
            examples["media_cn_cleaned"]
        )
    ]
    
    return model_tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=512 
    )

In [19]:
predict_dataset = predict_dataset.map(tokenize_three_columns, batched=True)
columns_to_remove = ['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']
predict_dataset = predict_dataset.remove_columns(columns_to_remove)
predict_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask'])


Map: 100%|██████████| 14370/14370 [00:03<00:00, 4414.82 examples/s]


In [ ]:
# def tokenize_fn_for_prediction(ex):
#     # label 칼럼이 없으므로 text 컬럼만 토큰화 진행
#     return model1_tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128) # max_length 조정 (128->512)

# predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
# predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"]) # 라벨은 없으므로 제외

In [ ]:
### roberta 사용 시 token_type_ids가 추가로 있어야 함

# def tokenize_fn_for_prediction(ex):
#     tokenized_output = model2_tokenizer(ex["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)
#     if "token_type_ids" not in tokenized_output:
#         tokenized_output["token_type_ids"] = [[0] * len(ids) for ids in tokenized_output["input_ids"]]
#     return tokenized_output

# predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
# predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", token_type_ids""]) # 라벨은 없으므로 제외

Map: 100%|██████████| 18981/18981 [00:01<00:00, 12261.69 examples/s]


In [ ]:
prediction_args = TrainingArguments(
    # output_dir="./prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False, # 학습은 안 함
    do_predict=True, # 예측만 수행
    report_to="none", # W&B 등 로깅 비활성화
    disable_tqdm=False, # 진행률 바 표시 (기본값 True)
)

# Trainer 인스턴스 생성
trainer = Trainer(model=model, args=prediction_args) # 모델 변경

# 예측 수행
predictions_output = trainer.predict(predict_dataset)

# predictions_output 객체에서 로짓 추출
logits = predictions_output.predictions
print(f"\n예측된 로짓 형태: {logits.shape}")

# 로짓을 확률로 변환 (필요한 경우)
probabilities = torch.softmax(torch.tensor(logits), dim=-1).numpy()

# 가장 높은 확률을 가진 라벨의 인덱스 추출
predicted_class_indices = np.argmax(logits, axis=-1)

# 라벨 인덱스를 실제 라벨 이름으로 매핑
category_labels = [f'{i}' for i in range(num_labels)]
predicted_label_names = [category_labels[idx] for idx in predicted_class_indices]


예측된 로짓 형태: (14370, 26)


In [24]:
new['predicted_label'] = predicted_label_names
new['predicted_label'] = new['predicted_label'].astype(int)

In [25]:
probs_df = pd.DataFrame(probabilities, columns=[f'prob_{label}' for label in category_labels])
new = pd.concat([new.reset_index(drop=True), probs_df.reset_index(drop=True)], axis=1)
new

,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned,predicted_label,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,...,prob_16,prob_17,prob_18,prob_19,prob_20,prob_21,prob_22,prob_23,prob_24,prob_25
0,yjmgoslow,yjm,yjm will drop a new album in the hottest summer,16,0.002786,0.007358,0.003483,0.010550,0.001860,0.012073,...,0.468594,0.015325,0.008815,0.170553,0.002231,0.011930,0.003620,0.006244,0.005798,0.001646
1,yjmgoslow,yjm,yg,19,0.000232,0.000165,0.000288,0.000441,0.000506,0.000460,...,0.000259,0.000781,0.000318,0.983935,0.000239,0.006725,0.000145,0.000470,0.000469,0.000190
2,yjmgoslow,yjm,저의 근황은더보기,19,0.000172,0.000131,0.000203,0.000350,0.000259,0.000548,...,0.000206,0.000723,0.000207,0.989646,0.000156,0.003760,0.000099,0.000315,0.000317,0.000164
3,yjmgoslow,yjm,99년식 김윤호 생일 축하해 주셔서 다들 감사해요,19,0.000497,0.000658,0.010724,0.000958,0.002319,0.001174,...,0.004702,0.001867,0.003472,0.722453,0.001677,0.003726,0.000671,0.002762,0.016437,0.003005
4,yjmgoslow,yjm,yoooooooo,19,0.000247,0.000170,0.000279,0.000497,0.000495,0.000475,...,0.000249,0.000853,0.000315,0.983531,0.000241,0.006814,0.000154,0.000521,0.000463,0.000194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14365,kikunionni,키쿠니온니 오전엔 직장인 오후엔 초보사장 o 3잡러의 일상 기록,오늘도 너무 더워요 자르고 또 자르고 요즘 중 단발이 유행이라 다시 시원하게 컷 해...,19,0.000116,0.000125,0.000277,0.000211,0.000429,0.000483,...,0.000395,0.000444,0.000321,0.985834,0.000162,0.001350,0.000147,0.003348,0.000640,0.000193
14366,kikunionni,키쿠니온니 오전엔 직장인 오후엔 초보사장 o 3잡러의 일상 기록,요즘 정말 덥지 않나요 녹아내릴 것 같은 날씨 저는 퇴근하고 시원하게 미술작품 보고...,19,0.000464,0.000400,0.000698,0.000740,0.000401,0.001131,...,0.000919,0.002073,0.000875,0.973563,0.000313,0.001220,0.000393,0.001187,0.001381,0.000515
14367,kikunionni,키쿠니온니 오전엔 직장인 오후엔 초보사장 o 3잡러의 일상 기록,인님들 사실 저 6월부터 새로운 직장으로 이직했어요 경기도민이 서울로 출근하는 건 ...,19,0.000105,0.000076,0.000229,0.000328,0.000369,0.000318,...,0.000160,0.000547,0.000225,0.993903,0.000088,0.000592,0.000073,0.000392,0.000406,0.000102
14368,kikunionni,키쿠니온니 오전엔 직장인 오후엔 초보사장 o 3잡러의 일상 기록,일만 하지 말고 잠깐 시간 내어 놀기도 해야죠 자꾸 같이 다니고 싶다던 사장님 저를...,19,0.000125,0.000096,0.000190,0.000324,0.000296,0.000425,...,0.000176,0.000612,0.000268,0.993492,0.000087,0.000760,0.000074,0.000353,0.000437,0.000097


##### ensemble predict

In [ ]:
# --- 0.4. 각 모델에 맞는 토크나이저 로드 및 텍스트 토큰화 ---

# kykim 모델용 토크나이저 및 데이터셋
model_name_1 = "kykim/bert-kor-base"
model_path_1 = "finetune-bert-kor"
model1 = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
model1.eval()

tokenizer_kykim = AutoTokenizer.from_pretrained(model_name_1)
def tokenize_fn_kykim_for_predict(examples):
    return tokenizer_kykim(examples["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)

predict_dataset_kykim_tokenized = predict_dataset.map(tokenize_fn_kykim_for_predict, batched=True)
predict_dataset_kykim_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])

# KoSimCSE 모델용 토크나이저 및 데이터셋
model_name_2 = "BM-K/KoSimCSE-roberta"
model_path_2 = "finetune-BM-K"

model2 = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
model2.eval()

tokenizer_kosimcse = AutoTokenizer.from_pretrained(model_name_2)
def tokenize_fn_kosimcse_for_predict(examples):
    tokenized_output = tokenizer_kosimcse(examples["media_cn_cleaned"], padding="max_length", truncation=True, max_length=128)
    if "token_type_ids" not in tokenized_output:
        tokenized_output["token_type_ids"] = [[0] * len(ids) for ids in tokenized_output["input_ids"]]
    return tokenized_output

predict_dataset_kosimcse_tokenized = predict_dataset.map(tokenize_fn_kosimcse_for_predict, batched=True)
predict_dataset_kosimcse_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids"])

# --- 2. Trainer를 사용하여 각 모델의 예측 로짓(Logits) 얻기 ---
# 더미 TrainingArguments (예측용이므로 대부분의 인자는 중요하지 않음)
prediction_args = TrainingArguments(
    output_dir="./voting_prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False,
    do_predict=True,
    report_to="none", # W&B 등 로깅 비활성화
    remove_unused_columns=False
)

# Model 1 Prediction
trainer1 = Trainer(model=model1, args=prediction_args)
predictions1 = trainer1.predict(predict_dataset_kykim_tokenized)
logits1 = predictions1.predictions # numpy array 형태의 로짓

# Model 2 Prediction
trainer2 = Trainer(model=model2, args=prediction_args)
predictions2 = trainer2.predict(predict_dataset_kosimcse_tokenized)
logits2 = predictions2.predictions


Map: 100%|██████████| 18282/18282 [00:01<00:00, 12330.75 examples/s]


In [ ]:
# --- 3. 로짓(Logits) 또는 확률(Probabilities) 평균내기 ---
# 소프트 보팅은 로짓 또는 softmax 확률을 평균낼 수 있음 (softmax는 비선형 변환이므로 평균 내기 전에 로짓 상태에서 평균 내는 것이 좋음)
# 일반적으로 로짓을 평균내는 것이 더 안정적

# 로짓 평균
ensemble_logits = (logits1 + logits2) / 2 
print(f"\n예측된 로짓 형태: {ensemble_logits.shape}")

# 로짓을 확률로 변환 (필요한 경우)
probabilities = torch.softmax(torch.tensor(ensemble_logits), dim=-1).numpy()

# 평균 로짓에서 최종 예측 클래스 선택
ensemble_predictions = np.argmax(ensemble_logits, axis=-1)

# 라벨 인덱스를 실제 라벨 이름으로 매핑
category_labels = [f'{i}' for i in range(num_labels)]
predicted_label_names = [category_labels[idx] for idx in ensemble_predictions]


예측된 로짓 형태: (18282, 26)


In [19]:
new['predicted_label'] = predicted_label_names
new['predicted_label'] = new['predicted_label'].astype(int)

In [26]:
label_map = {
 'IT': 0,
 '게임': 1,
 '결혼/연애': 2,
 '교육': 3,
 '다이어트/건강보조식품': 4,
 '만화/애니/툰': 5,
 '문구/완구': 6,
 '미술/디자인': 7,
 '반려동물': 8,
 '베이비/키즈': 9,
 '뷰티': 10,
 '브랜드공식계정': 11,
 '사진/영상': 12,
 '셀럽': 13,
 '스포츠': 14,
 '시사': 15,
 '엔터테인먼트': 16,
 '여행/관광': 17,
 '유명장소/핫플': 18,
 '일상': 19,
 '자동차/모빌리티': 20,
 '짤/밈': 21,
 '취미': 22,
 '패션': 23,
 '푸드': 24,
 '홈/리빙': 25
}

inv_label_map = {v: k for k, v in label_map.items()}
new['single_label'] = new['predicted_label'].map(inv_label_map)

In [ ]:
# new.to_excel("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/new_multi-columns.xlsx")